In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv
import edurel.utils.question_man as qm
import edurel.utils.conversation as conv
import edurel.utils.duckdb as ddb
import edurel.utils.sql_agent as sa


load_dotenv() 
BASE_DIR = os.getenv("BASE_DIR")
QUESTIONS_DIR = f"{BASE_DIR}/questions"
CONFIG_DIR = f"{BASE_DIR}/config"


/home/iclassen/.py1/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


# Database

In [3]:
db = ddb.Db.mem("company_en")
db.config_exe(f"{CONFIG_DIR}/databases/company_en.yaml")

# Questions

In [5]:
qman = qm.QuestionMan(f"{QUESTIONS_DIR}/company_en.txt")

In [6]:
sqlagent = sa.SQLAgent(
    db,
    conv.Client.STATS, 
    conv.LLMName.GLM47,
    fk_spec_file_path=f"{CONFIG_DIR}/fk_specs/company_en.yaml",
    rem_tags_spec_file_path=f"{CONFIG_DIR}/omit_tags_specs/dbstuff.yaml",
    log_dir=f"{BASE_DIR}/logs",
    autolog=False
)

# Text2SQL

In [7]:
sqlagent.text2sql(qman[0][0], qman[0][1])

In [8]:
sqlagent.conversation.display(lastn_only=3)

[2] USER:

create a SQL query that outputs the following columns:
- eid, lastname, hiredate from table employee
- entry year per employee 

sorted by eid

[3] AI:

```sql
SELECT 
    EID, 
    LastName, 
    Hiredate, 
    EXTRACT(YEAR FROM Hiredate) AS entry_year
FROM Employee
ORDER BY EID
```

[4] USER:

```plaintext
┌───────┬──────────┬────────────┬────────────┐
│  EID  │ LastName │  Hiredate  │ entry_year │
│ int32 │ varchar  │    date    │   int64    │
├───────┼──────────┼────────────┼────────────┤
│   101 │ Patil    │ 2000-05-01 │       2000 │
│   102 │ Durmaz   │ 2005-07-01 │       2005 │
│   103 │ Blaschke │ 2002-11-01 │       2002 │
│   104 │ Stone    │ 2006-06-01 │       2006 │
│   105 │ Dalal    │ 2018-02-02 │       2018 │
│   106 │ Li       │ 2002-12-01 │       2002 │
│   107 │ Nguyen   │ 2006-07-01 │       2006 │
│   108 │ Sanchez  │ 2014-04-01 │       2014 │
│   109 │ Umarani  │ 2006-07-01 │       2006 │
│   110 │ Ortega   │ 2005-09-02 │       2005 │
│   111 │ Doshi    │ 2010-01-02 │       2010 │
│   112 │ Singh    │ 2012-03-01 │       2012 │
│   113 │ Jadhav   │ 2001-08-01 │       2001 │
│   114 │ Popov    │ 2009-03-02 │       2009 │
│   115 │ Kumar    │ 2013-05-01 │       2013 │
│   116 │ Krause   │ 2011-08-01 │       2011 │
│   117 │ Oezdem   │ 2014-08-01 │       2014 │
│   118 │ Okeke    │ 2013-11-01 │       2013 │
├───────┴──────────┴────────────┴────────────┤
│ 18 rows                          4 columns │
└────────────────────────────────────────────┘

```

In [10]:
sqlagent.log(qman[0][0], "v1")

In [14]:
print(qman[0][1])

create a SQL query that outputs the following columns:
- eid, lastname, hiredate from table employee
- entry year per employee 

sorted by eid


In [4]:
for q in qman[:]:
    print(f"---tag: {q[0]}")
    print(q[1])

---tag: entry_year
create a SQL query that outputs the following columns:
- eid, lastname, hiredate from table employee
- entry year per employee 
sorted by eid
---tag: entry_year_dept_14
create a SQL query that outputs the following columns:
- eid, lastname, hiredate from table employee
- entry year per employee 
only for department 14
sorted by eid
---tag: hireyears_dept_14
create a SQL query that outputs the following columns:
- eid, lastname, hiredate from table employee
- number of years of deployment per employee 
only for department 14
sorted by eid
---tag: employees_without_bonus
create a SQL query that outputs the following columns:
- lastname, salary, bonus from table employee
only employees without bonus
sorted by eid
---tag: employees_with_bonus
create a SQL query that outputs the following columns:
- lastname, salary, bonus from table employee
only employees with bonus
sorted by eid
---tag: total_income_per_employe_dept_17
create a SQL query that outputs the following colu